In [ ]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv("doctor_train.csv")

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.keys

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['Miscellaneous_Info'],axis=1, inplace= True)

In [ ]:
df['Rating'].fillna('0%',inplace=True)

In [ ]:
df['Place'].fillna(df['Place'].mode()[0], inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
sorted(df.Qualification[df.Qualification.apply(lambda x: len(x.split(','))).idxmax()].split(","))

In [ ]:
import re

def sortQualification(text):
    arr =re.sub(r'\([^()]+\)',lambda x:x.group().replace(",","-"),text)
    return ','.join(sorted(arr.lower().replace(" ","").split(",")))

In [ ]:
df.Qualification= df.Qualification.apply(lambda x: sortQualification(x))

In [ ]:
df.Qualification

In [ ]:
df["Experience"]=df["Experience"].str.split()
df["Experience"]= df["Experience"].str[0].astype("int")

In [ ]:
df["Rating"]= df["Rating"].str[:-1].astype("int")

In [ ]:
df

In [ ]:
plt.figure(figsize=(20,20))
sns.barplot(x='Experience',y='Fees',data=df)
plt.title("Doctor Fee")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
sns.barplot(x='Rating',y='Fees',data=df)
plt.title("Doctor Fee")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
sns.barplot(x='Profile',y='Fees',data=df)
plt.title("Doctor Fee")
plt.xticks(rotation=45)
plt.show()

In [ ]:
for col in df.select_dtypes(['object']).columns: #Checking categorical data
    print(col, ':', sorted(df[col].unique()))

In [ ]:
var_cat = df.select_dtypes(include=[object])
var_cat.head()

In [ ]:
var_cat = var_cat.columns.tolist()
var_cat = ['Qualification','Place','Profile']
var_cat
['Qualification','Place','Profile']

In [ ]:
from sklearn.preprocessing import LabelEncoder #Converting catagorical data into numerical
LE=LabelEncoder()

df["Qualification"]=LE.fit_transform(df["Qualification"])
df["Place"]=LE.fit_transform(df["Place"])
df["Profile"]=LE.fit_transform(df["Profile"])

In [ ]:
dfcor=df.corr() 
dfcor

In [ ]:
plt.figure(figsize=(16,14))
sns.heatmap(dfcor,annot=True)
plt.title("Correlation Matrix")
plt.show()

In [ ]:
corr_matrix=df.corr()

In [ ]:
corr_matrix["Fees"].sort_values(ascending=False)

In [ ]:
df.plot(kind='box',subplots=True, layout=(6,4), figsize= (20,15))

In [ ]:
#removing outlieers
from scipy.stats import zscore
z=np.abs(zscore(df))
z

In [ ]:
threshold=3
print(np.where(z>3))

In [ ]:
df_new=df[(z<3).all(axis=1)]
print(df.shape)
print(df_new.shape)

In [ ]:
loss_percent=(5961-5903)/5961*100
print(loss_percent)

In [ ]:
sns.pairplot(df)
plt.show()

In [ ]:
df.skew() # Checking Skewness

In [ ]:
skew=("Experience","Fees")
for col in skew :
    if df.skew().loc[col]>0.55:
        df[col]=np.log1p(df[col])

In [ ]:
df.skew()

In [ ]:
x=df.drop("Fees",axis=1)
y=df["Fees"]

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)
x

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [ ]:
def score(y_pred,y):
    y_pred = np.log(y_pred)
    y = np.log(y)
    return 1 - ((np.sum((y_pred-y)**2))/len(y))**1/2


In [ ]:
knn = KNeighborsRegressor()
svm = SVR(gamma='scale')
dt = DecisionTreeRegressor()
rf = RandomForestRegressor()

In [ ]:
for m in (knn, svm, dt, rf):
    m.fit(x_train, y_train)
    
    y_pred = m.predict(x_test)
    
    print(m, score(y_pred,y_test))

In [ ]:
n_estimators = [int(x) for x in np.linspace(start=150,stop=1000,num=6)]
#max levels in tree
max_depth = [int(x) for x in np.linspace(start=6,stop=30,num=5)]
#min no. of splitting required to split a node
min_samples_split = [2,7,10]
#min no. of sample required at each leaf node
min_samples_leaf = [2,5]
#max_features
max_features=['sqrt','auto']

# generate a dictionary of all the Hyper Parameters
random_grid = {'n_estimators':n_estimators,'max_depth':max_depth,'min_samples_split':min_samples_split,'min_samples_leaf': min_samples_leaf,'max_features':max_features,}